In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#pip install --upgrade ipykernel

In [3]:
#pip install autocorrect


In [4]:
#pip install textblob


In [5]:
#pip install gensim==3.8.3


In [6]:
#!pip install pyLDAvis


In [7]:
#pip install spacy


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import nltk
import scipy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from autocorrect import Speller
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from gensim.models import CoherenceModel
import gensim
import gensim.corpora as corpora
import spacy

In [9]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [10]:
!type -a pip

pip is /Library/Frameworks/Python.framework/Versions/3.9/bin/pip


In [11]:
import pyLDAvis
import pyLDAvis.gensim_models

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [12]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/janine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
merged = pd.read_csv("merged_new.csv")
merged.Content

0     This paper aims to compute the performances of...
1     Renewable energy project development is highly...
2     This study assesses the factors responsible fo...
3     An empirical analysis of the non-linear impact...
4     Climate change is increasingly recognized worl...
                            ...                        
82    This paper analyses the politics of renewable ...
83    Expansion of renewable energies is a central p...
84    Stakeholders strongly influence project succes...
85    This paper aims to analyse reasons for the abs...
86    Citizens own nearly half the renewable energy ...
Name: Content, Length: 87, dtype: object

In [14]:
#display word count
merged['word_count'] = merged['Content'].apply(lambda x: len(str(x).split(" ")))
merged[['Content','word_count']].head()

,Content,word_count
0,This paper aims to compute the performances of...,777
1,Renewable energy project development is highly...,768
2,This study assesses the factors responsible fo...,817
3,An empirical analysis of the non-linear impact...,27
4,Climate change is increasingly recognized worl...,490


# Data prep

In [15]:
#lowercase content
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
merged['Content'].head()

0    this paper aims to compute the performances of...
1    renewable energy project development is highly...
2    this study assesses the factors responsible fo...
3    an empirical analysis of the non-linear impact...
4    climate change is increasingly recognized worl...
Name: Content, dtype: object

In [16]:
#remove punctuations
merged['Content'] = merged['Content'].str.replace('[^\w\s]','')
merged['Content'].head()

<>:2: DeprecationWarning: invalid escape sequence \w
<>:2: DeprecationWarning: invalid escape sequence \w
/var/folders/0j/p1sf5x296z57_lsn25m7m2y80000gn/T/ipykernel_24958/509571761.py:2: DeprecationWarning: invalid escape sequence \w
  merged['Content'] = merged['Content'].str.replace('[^\w\s]','')


0    this paper aims to compute the performances of...
1    renewable energy project development is highly...
2    this study assesses the factors responsible fo...
3    an empirical analysis of the nonlinear impacts...
4    climate change is increasingly recognized worl...
Name: Content, dtype: object

In [17]:
#display number of stopwords

from nltk.corpus import stopwords
stop = stopwords.words('english')

merged['stopwords'] = merged['Content'].apply(lambda x: len([x for x in x.split() if x in stop]))
merged[['Content','stopwords']].head()

,Content,stopwords
0,this paper aims to compute the performances of...,319
1,renewable energy project development is highly...,315
2,this study assesses the factors responsible fo...,277
3,an empirical analysis of the nonlinear impacts...,7
4,climate change is increasingly recognized worl...,179


In [18]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
#remove stopwords
stop = stopwords.words('english')
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()

0    paper aims compute performances smaller versio...
1    renewable energy project development highly co...
2    study assesses factors responsible success fai...
3    empirical analysis nonlinear impacts icttrade ...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

# Exploratory analysis 

In [20]:
TextBlob(merged['Content'][0]).ngrams(2)

[WordList(['paper', 'aims']),
 WordList(['aims', 'compute']),
 WordList(['compute', 'performances']),
 WordList(['performances', 'smaller']),
 WordList(['smaller', 'version']),
 WordList(['version', 'solana']),
 WordList(['solana', 'power']),
 WordList(['power', 'plant']),
 WordList(['plant', 'half']),
 WordList(['half', 'solar']),
 WordList(['solar', 'field']),
 WordList(['field', '1']),
 WordList(['1', '2']),
 WordList(['2', 'turbines']),
 WordList(['turbines', 'power']),
 WordList(['power', 'cycle']),
 WordList(['cycle', 'built']),
 WordList(['built', 'amman']),
 WordList(['amman', 'maan']),
 WordList(['maan', 'jordan']),
 WordList(['jordan', 'climate']),
 WordList(['climate', 'conditions']),
 WordList(['conditions', 'amman']),
 WordList(['amman', 'maan']),
 WordList(['maan', 'discussed']),
 WordList(['discussed', 'thoroughly']),
 WordList(['thoroughly', 'paper']),
 WordList(['paper', 'furthermore']),
 WordList(['furthermore', 'preliminary']),
 WordList(['preliminary', 'validation']

In [21]:
TextBlob(merged['Content'][0]).ngrams(3)

[WordList(['paper', 'aims', 'compute']),
 WordList(['aims', 'compute', 'performances']),
 WordList(['compute', 'performances', 'smaller']),
 WordList(['performances', 'smaller', 'version']),
 WordList(['smaller', 'version', 'solana']),
 WordList(['version', 'solana', 'power']),
 WordList(['solana', 'power', 'plant']),
 WordList(['power', 'plant', 'half']),
 WordList(['plant', 'half', 'solar']),
 WordList(['half', 'solar', 'field']),
 WordList(['solar', 'field', '1']),
 WordList(['field', '1', '2']),
 WordList(['1', '2', 'turbines']),
 WordList(['2', 'turbines', 'power']),
 WordList(['turbines', 'power', 'cycle']),
 WordList(['power', 'cycle', 'built']),
 WordList(['cycle', 'built', 'amman']),
 WordList(['built', 'amman', 'maan']),
 WordList(['amman', 'maan', 'jordan']),
 WordList(['maan', 'jordan', 'climate']),
 WordList(['jordan', 'climate', 'conditions']),
 WordList(['climate', 'conditions', 'amman']),
 WordList(['conditions', 'amman', 'maan']),
 WordList(['amman', 'maan', 'discussed

In [22]:
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

energy           1174
renewable         684
policy            387
project           294
success           280
projects          256
factors           249
development       223
also              222
study             205
important         198
research          193
community         170
technology        169
support           164
local             152
different         148
stakeholders      136
environmental     135
policies          135
dtype: int64

In [23]:
#add new stop words

stop = stopwords.words('english')
newStopWords = ['also','1','2','3','4','turkey','et','al','csfs','different','jordan','maan','anman','important','results','however','paper','new','implications','research','may','factors','study','success','this''csf','project','finding','result','decision','impact''important','solar','wind','renewable','energy','factors','projects','success','study']
stop.extend(newStopWords)

merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()


0    aims compute performances smaller version sola...
1    development highly complex means guaranteed de...
2    assesses responsible failure access programs b...
3    empirical analysis nonlinear impacts icttrade ...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

In [24]:
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

policy           387
development      223
community        170
technology       169
support          164
local            152
stakeholders     136
environmental    135
policies         135
analysis         130
financial        124
government       123
role             120
economic         118
technologies     114
sector           113
social           109
one              104
power            103
use              101
dtype: int64

In [25]:
TextBlob(merged['Content'][0]).ngrams(2)

[WordList(['aims', 'compute']),
 WordList(['compute', 'performances']),
 WordList(['performances', 'smaller']),
 WordList(['smaller', 'version']),
 WordList(['version', 'solana']),
 WordList(['solana', 'power']),
 WordList(['power', 'plant']),
 WordList(['plant', 'half']),
 WordList(['half', 'field']),
 WordList(['field', 'turbines']),
 WordList(['turbines', 'power']),
 WordList(['power', 'cycle']),
 WordList(['cycle', 'built']),
 WordList(['built', 'amman']),
 WordList(['amman', 'climate']),
 WordList(['climate', 'conditions']),
 WordList(['conditions', 'amman']),
 WordList(['amman', 'discussed']),
 WordList(['discussed', 'thoroughly']),
 WordList(['thoroughly', 'furthermore']),
 WordList(['furthermore', 'preliminary']),
 WordList(['preliminary', 'validation']),
 WordList(['validation', 'exercise']),
 WordList(['exercise', 'performed']),
 WordList(['performed', 'using']),
 WordList(['using', 'measured']),
 WordList(['measured', 'monthly']),
 WordList(['monthly', 'average']),
 WordList

In [26]:
TextBlob(merged['Content'][0]).ngrams(3)

[WordList(['aims', 'compute', 'performances']),
 WordList(['compute', 'performances', 'smaller']),
 WordList(['performances', 'smaller', 'version']),
 WordList(['smaller', 'version', 'solana']),
 WordList(['version', 'solana', 'power']),
 WordList(['solana', 'power', 'plant']),
 WordList(['power', 'plant', 'half']),
 WordList(['plant', 'half', 'field']),
 WordList(['half', 'field', 'turbines']),
 WordList(['field', 'turbines', 'power']),
 WordList(['turbines', 'power', 'cycle']),
 WordList(['power', 'cycle', 'built']),
 WordList(['cycle', 'built', 'amman']),
 WordList(['built', 'amman', 'climate']),
 WordList(['amman', 'climate', 'conditions']),
 WordList(['climate', 'conditions', 'amman']),
 WordList(['conditions', 'amman', 'discussed']),
 WordList(['amman', 'discussed', 'thoroughly']),
 WordList(['discussed', 'thoroughly', 'furthermore']),
 WordList(['thoroughly', 'furthermore', 'preliminary']),
 WordList(['furthermore', 'preliminary', 'validation']),
 WordList(['preliminary', 'valid

In [27]:
merged=merged.drop(['Title','word_count','stopwords'],axis =1)


In [28]:
merged

,Content
0,aims compute performances smaller version sola...
1,development highly complex means guaranteed de...
2,assesses responsible failure access programs b...
3,empirical analysis nonlinear impacts icttrade ...
4,climate change increasingly recognized worldwi...
...,...
82,analyses politics discussing opportunities wel...
83,expansion energies central pillar german trans...
84,stakeholders strongly influence particularly c...
85,aims analyse reasons absence electricity impor...


# Data prep

In [29]:
#tokenization
TextBlob(merged['Content'][1]).words

#lemmatization
from textblob import Word
merged['Content'] = merged['Content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
merged['Content'].head()

0    aim compute performance smaller version solana...
1    development highly complex mean guaranteed dec...
2    ass responsible failure access program banglad...
3    empirical analysis nonlinear impact icttrade o...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

# LDA Model - Gensim
#source: 
https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/

In [30]:
merged_content = [text.split(" ") for text in merged.Content]
merged_content

[['aim',
  'compute',
  'performance',
  'smaller',
  'version',
  'solana',
  'power',
  'plant',
  'half',
  'field',
  'turbine',
  'power',
  'cycle',
  'built',
  'amman',
  'climate',
  'condition',
  'amman',
  'discussed',
  'thoroughly',
  'furthermore',
  'preliminary',
  'validation',
  'exercise',
  'performed',
  'using',
  'measured',
  'monthly',
  'average',
  'value',
  'electricity',
  'production',
  'existing',
  'plant',
  'system',
  'advisor',
  'model',
  'sam',
  'used',
  'predict',
  'performance',
  'proposed',
  'solanalike',
  'plant',
  'amman',
  'validation',
  'show',
  'good',
  'agreement',
  'measured',
  'data',
  'existing',
  'power',
  'plant',
  'simulation',
  'including',
  'monthly',
  'capacity',
  'suggest',
  'annual',
  'operation',
  'maybe',
  'even',
  'better',
  'operation',
  'gila',
  'bend',
  'annual',
  'average',
  'capacity',
  'factor',
  '41',
  'v',
  'capacity',
  'factor',
  '39',
  'gila',
  'bend',
  'mainly',
  'due',

In [31]:
# only keep words above 10 counts
from gensim.corpora.dictionary import Dictionary
dictionary = Dictionary(merged_content)
dictionary.filter_extremes(no_below=10)

In [32]:
# Create Dictionary
id2word = corpora.Dictionary(merged_content)
# Create Corpus
texts = merged_content
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 7), (28, 4), (29, 1), (30, 1), (31, 1), (32, 4), (33, 4), (34, 1), (35, 6), (36, 1), (37, 1), (38, 3), (39, 1), (40, 12), (41, 2), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 4), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 12), (58, 1), (59, 1), (60, 3), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 3), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 2), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 2), (82, 5), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 3), (97, 2), (98, 1), (99, 2), (100, 1), (101, 1), (102, 1), (103, 3), (104, 1), (105, 1), (106, 4), (107, 1), (108, 1), (109, 2), (110, 

In [33]:
## create LDA model
from gensim.models.ldamodel import LdaModel
lda_model = LdaModel(corpus=corpus, id2word=id2word,
                    num_topics=5, random_state=100,
                    chunksize=100, passes=10,
                     per_word_topics=True)
from pprint import pprint
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus] 

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=merged_content, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Compute Coherence Score using UMass
coherence_Um_model_lda = CoherenceModel(model=lda_model, texts=merged_content, dictionary=dictionary, coherence="u_mass")
coherence_Um_lda = coherence_Um_model_lda.get_coherence()
print('\nUMassCoherence Score: ', coherence_Um_lda)

## presentation -- final graph or word cloud per topic
## repeating words - see the LDA visualisation - it doesnt match the index here. 
## see topic 0 and 1 are well seperated - new random numbering, look at the top few words to match the grouping.
## add another model - final report - how well seperated the topics are.. clustering. see LSA and PLSA

[(0,
  '0.014*"policy" + 0.009*"investment" + 0.009*"community" + 0.007*"financial" '
  '+ 0.007*"performance" + 0.007*"investor" + 0.006*"system" + '
  '0.006*"institutional" + 0.006*"technology" + 0.006*"measure"'),
 (1,
  '0.017*"policy" + 0.007*"technology" + 0.007*"industry" + 0.004*"government" '
  '+ 0.004*"environmental" + 0.004*"adoption" + 0.004*"level" + 0.004*"role" + '
  '0.004*"support" + 0.004*"development"'),
 (2,
  '0.010*"policy" + 0.009*"stakeholder" + 0.009*"community" + '
  '0.008*"development" + 0.006*"process" + 0.005*"local" + 0.005*"management" '
  '+ 0.004*"support" + 0.004*"government" + 0.004*"technology"'),
 (3,
  '0.008*"technology" + 0.007*"policy" + 0.006*"system" + 0.005*"development" '
  '+ 0.005*"community" + 0.005*"traceability" + 0.005*"group" + 0.004*"social" '
  '+ 0.004*"support" + 0.004*"case"'),
 (4,
  '0.009*"state" + 0.007*"policy" + 0.006*"technology" + 0.005*"investor" + '
  '0.005*"expansion" + 0.005*"government" + 0.005*"plant" + 0.005*"c

# LDA Mallet

In [34]:
#use Mallet to improve LDA cohrence score.
mallet_path = '/Users/janine/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=merged_content, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Mallet LDA: 5 topics, 3 topic bits, 111 topic mask
Data loaded.
max tokens: 1432
total tokens: 35906
<10> LL/token: -8.94105
<20> LL/token: -8.5965
<30> LL/token: -8.46676
<40> LL/token: -8.4215

0	10	process role literature system environmental technology relationship institutional framework sustainability context environment actor traceability supply analysis study effective consumer information 
1	10	policy investment industry target change level emission china country climate instrument environmental market support international model article investor analysis global 
2	10	technology financial cost adoption capacity performance measure plant finding impact benefit provide potential subsidy positive economic design individual term significant 
3	10	development sector government state power electricity source production policy economic transition renewables system sustainable barrier level year public data country 
4	10	community stakeholder local social support case management netwo

<410> LL/token: -8.30345
<420> LL/token: -8.30521
<430> LL/token: -8.30606
<440> LL/token: -8.3098

0	10	system environmental role literature technology process sustainability driver partnership analysis information approach traceability supply work study consumer critical field management 
1	10	policy investment industry level investor potential country public emission china future climate target international instrument finding global sustainable market make 
2	10	technology cost adoption impact capacity performance measure plant financial economic show time positive program benefit significant private importance affect access 
3	10	policy development sector state government power support electricity source production country renewables year market political economic germany barrier initiative region 
4	10	community stakeholder local process social support network development case management based financial area change model organization member individual opportunity construction 

<

<810> LL/token: -8.29731
<820> LL/token: -8.29721
<830> LL/token: -8.29749
<840> LL/token: -8.30393

0	10	literature environmental technology system study change role process approach partnership framework sustainability driver analysis government information traceability supply implementation management 
1	10	policy investment industry investor potential level public emission transition china climate case sustainable international future global trade specific risk issue 
2	10	technology cost adoption impact financial capacity performance measure plant benefit economic system positive program significant importance time effect affect access 
3	10	policy development sector state country support power electricity source production government market renewables barrier economic political germany environmental initiative region 
4	10	community stakeholder local social process support based development network management case organization resource government cre opportunity technical structu

[(0,
  [('environmental', 0.015190642564180465),
   ('system', 0.014431110435971442),
   ('literature', 0.0130639526051952),
   ('technology', 0.011544888348777153),
   ('role', 0.010025824092359107),
   ('process', 0.009570104815433692),
   ('framework', 0.009418198389791889),
   ('sustainability', 0.00881057268722467),
   ('partnership', 0.00820294698465745),
   ('analysis', 0.0074434148564484276)]),
 (1,
  [('policy', 0.045318512184694314),
   ('investment', 0.023371811315376943),
   ('industry', 0.016246259085079092),
   ('level', 0.014251104460595696),
   ('investor', 0.013538549237565912),
   ('potential', 0.011970927746900385),
   ('target', 0.01083083939005273),
   ('public', 0.009975773122416987),
   ('emission', 0.00983326207781103),
   ('transition', 0.009690751033205073)]),
 (2,
  [('technology', 0.031203120312031204),
   ('cost', 0.014251425142514252),
   ('adoption', 0.01395139513951395),
   ('performance', 0.01155115511551155),
   ('impact', 0.010351035103510351),
   ('m

# LDA Gensim Visualizations

In [35]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.009344  0.015803       1        1  29.923775
2      0.078400 -0.002189       2        1  26.349733
3      0.047198  0.036238       3        1  22.155047
4     -0.037142 -0.108179       4        1  11.186714
0     -0.097801  0.058327       5        1  10.384731, topic_info=               Term        Freq       Total Category  logprob  loglift
1447          state  105.000000  105.000000  Default  30.0000  30.0000
361        investor   79.000000   79.000000  Default  29.0000  29.0000
500       community  212.000000  212.000000  Default  28.0000  28.0000
596      investment  140.000000  140.000000  Default  27.0000  27.0000
392          policy  458.000000  458.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
515         country   15.203068  122.398123   Topic5  -5.6169   0.1791
309     development   15.418887  206.194549   Topic5  -5.6028  -0.3284
80         economic   13.993533  103.406549   Topic5  -5.6998   0.2648
83    environmental   13.252566  118.870392   Topic5  -5.7542   0.0710
785          effect   12.469440   68.769229   Topic5  -5.8151   0.5574

[387 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
3609      5  0.814829     001
3613      5  0.814853     058
1609      5  0.814817    1980
4807      2  0.807160    2020
4807      4  0.142440    2020
...     ...       ...     ...
448       4  0.047153  within
448       5  0.078588  within
1912      2  0.936371   woody
4448      4  0.972649     wtp
3702      5  0.884156       β

[806 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 5, 1])

In [ ]:
#making Coherence graph
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=merged_content, start=2, limit=15, step=1)


Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 1432
total tokens: 35906
<10> LL/token: -8.58769
<20> LL/token: -8.43682
<30> LL/token: -8.36759
<40> LL/token: -8.32743

0	25	technology stakeholder industry process power model change cost management adoption electricity sustainable study impact capacity importance target performance financial measure 
1	25	policy development community support government system local country sector state environmental economic social investment market investor case potential source production 

<50> LL/token: -8.3103
<60> LL/token: -8.29486
<70> LL/token: -8.29026
<80> LL/token: -8.27862
<90> LL/token: -8.27832

0	25	technology stakeholder industry process change cost model management adoption electricity sustainable study impact capacity importance target performance time measure plant 
1	25	policy development community support government local investment country sector environmental system state economic level social market f

<860> LL/token: -8.25088
<870> LL/token: -8.24823
<880> LL/token: -8.25265
<890> LL/token: -8.25482

0	25	technology stakeholder process system community model cost management adoption study literature impact capacity implementation performance analysis measure plant financial emission 
1	25	policy development investment local country sector environmental support government state community technology industry economic power market level investor potential electricity 

<900> LL/token: -8.25512
<910> LL/token: -8.25377
<920> LL/token: -8.25168
<930> LL/token: -8.24633
<940> LL/token: -8.24815

0	25	technology stakeholder process system community model cost management adoption study literature financial capacity implementation impact performance measure plant emission china 
1	25	policy development investment local country sector environmental state industry support economic community level market government technology investor power potential electricity 

<950> LL/token: -8.24744
<960>

<510> LL/token: -8.26869
<520> LL/token: -8.27203
<530> LL/token: -8.27756
<540> LL/token: -8.2753

0	16.66667	community process system stakeholder social local management implementation plant case development member sustainability literature importance driver partnership individual focus article 
1	16.66667	development government industry financial analysis model support capacity target level performance measure sustainable role emission show china relationship network power 
2	16.66667	policy technology investment environmental state economic market country investor potential adoption electricity sector source production effect framework public transition renewables 

<550> LL/token: -8.28171
<560> LL/token: -8.27792
<570> LL/token: -8.27684
<580> LL/token: -8.27389
<590> LL/token: -8.27602

0	16.66667	community process local system social stakeholder management implementation plant importance literature member sustainability driver partnership individual support case program article

<110> LL/token: -8.30624
<120> LL/token: -8.30897
<130> LL/token: -8.29904
<140> LL/token: -8.2938

0	12.5	policy development government community sector support power change level source analysis target effect public show instrument increase barrier economic growth 
1	12.5	system management process measure plant emission china variable importance member sustainability sustainable performance driver carbon efficiency time literature information traceability 
2	12.5	technology stakeholder social role adoption local community case model context framework process key group based relationship network influence implementation environmental 
3	12.5	policy investment country state industry market investor potential electricity production environmental transition renewables future climate institutional local capacity focus economic 

<150> LL/token: -8.294
<160> LL/token: -8.29197
<170> LL/token: -8.29549
<180> LL/token: -8.30091
<190> LL/token: -8.29251

0	12.5	policy development government s

<610> LL/token: -8.29894
<620> LL/token: -8.29211
<630> LL/token: -8.29015
<640> LL/token: -8.28986

0	12.5	policy development sector government power state level source target production economic analysis china technology support instrument change year national increase 
1	12.5	system stakeholder management process performance plant emission variable literature sustainability cost method driver carbon efficiency financial model information traceability supply 
2	12.5	technology community local role social support adoption based environmental key analysis study case context group relationship network process implementation model 
3	12.5	policy investment country industry investor future potential transition renewables market case environmental electricity international finding germany existing global impact condition 

<650> LL/token: -8.29104
<660> LL/token: -8.29432
<670> LL/token: -8.29066
<680> LL/token: -8.28937
<690> LL/token: -8.29179

0	12.5	policy development sector government

...

<310> LL/token: -8.31898
<320> LL/token: -8.31437
<330> LL/token: -8.31615
<340> LL/token: -8.31758

0	7.14286	technology adoption environmental performance measure framework financial impact role model finding global literature private understanding cost positive relationship term study 
1	7.14286	community local government social case support group analysis process role partnership context actor key successful based cre development number resource 
2	7.14286	stakeholder community process management sustainability variable opportunity construction technical literature critical influence development identified quality proposed enterprise good structure beneficiary 
3	7.14286	system method plant efficiency information traceability supply consumer review management driver analysis power issue implementation model product green member standard 
4	7.14286	investment state country market investor potential transition renewables institutional policy economic electricity capacity germany powe

<610> LL/token: -8.30346
<620> LL/token: -8.30758
<630> LL/token: -8.30604
<640> LL/token: -8.29397

0	7.14286	technology environmental adoption performance framework measure role impact literature finding cost financial global potential private effect sustainability focus economic future 
1	7.14286	community local social government case network group role partnership based support context development successful process cre actor interaction implementation number 
2	7.14286	stakeholder community model variable process management proposed relationship construction development critical technical quality influence enterprise key good beneficiary manager identified 
3	7.14286	system method plant information traceability supply consumer management issue implementation review driver barrier study impact efficiency analysis product power capacity 
4	7.14286	investment state country investor policy transition analysis renewables market economic capacity level existing initiative potential cond

<910> LL/token: -8.31222
<920> LL/token: -8.31478
<930> LL/token: -8.30907
<940> LL/token: -8.30601

0	7.14286	technology environmental adoption measure framework literature performance role financial global impact sustainability private economic context group finding positive knowledge suggest 
1	7.14286	community local social government support transition case role partnership development actor network number cre successful individual process uk interaction resource 
2	7.14286	stakeholder process model community management variable proposed construction relationship identified quality influence good critical beneficiary responsibility manager technical enterprise work 
3	7.14286	system method management issue information traceability supply plant consumer review implementation barrier study efficiency case cost product approach application chain 
4	7.14286	investment state investor policy market potential renewables country capacity germany institutional existing initiative strategy 

<210> LL/token: -8.34149
<220> LL/token: -8.34257
<230> LL/token: -8.34121
<240> LL/token: -8.33387

0	6.25	process action structure role cre traditional target government provide subsidy financial trm responsibility individual work plant roadmaps developing recommendation level 
1	6.25	investment investor emission policy sustainable carbon country risk china fit public fuel potential climate study reduction market analysis price problem 
2	6.25	community system local management transition sustainability implementation process literature initiative actor traceability resource organization consumer number activity review good supply 
3	6.25	technology stakeholder adoption model relationship analysis role variable influence environmental study partnership proposed finding importance building method literature based time 
4	6.25	policy industry social environmental context instrument member level support international specific local group analysis field mix interaction type technological 

<510> LL/token: -8.33223
<520> LL/token: -8.34676
<530> LL/token: -8.33433
<540> LL/token: -8.32591

0	6.25	process financial community support structure cre technical government quality traditional developing trm action responsibility level plant roadmaps recommendation opportunity work 
1	6.25	investment investor emission sustainable carbon potential study risk problem barrier reduction policy driver market china decision public target analysis make 
2	6.25	community system management local implementation transition organization initiative sustainability support traceability supply number consumer activity characteristic literature focus lead change 
3	6.25	technology stakeholder adoption relationship variable influence model partnership analysis proposed study finding building time network efficiency framework perception case role 
4	6.25	policy industry environmental context social instrument case level change role group institutional climate design field mix interaction based spec

In [ ]:
# Show graph
limit=15; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.title ("")
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=merged_content):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=merged_content)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(30)

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)



In [ ]:
doc_lens = [len(d) for d in df_dominant_topic.Text]

# Plot
plt.figure(figsize=(16,7), dpi=160)
plt.hist(doc_lens, bins = 1000, color='navy')
plt.text(750, 100, "Mean   : " + str(round(np.mean(doc_lens))))
plt.text(750,  90, "Median : " + str(round(np.median(doc_lens))))
plt.text(750,  80, "Stdev   : " + str(round(np.std(doc_lens))))
plt.text(750,  70, "1%ile    : " + str(round(np.quantile(doc_lens, q=0.01))))
plt.text(750,  60, "99%ile  : " + str(round(np.quantile(doc_lens, q=0.99))))

plt.gca().set(xlim=(0, 1000), ylabel='Number of Documents', xlabel='Document Word Count')
plt.tick_params(size=16)
plt.xticks(np.linspace(0,1000,9))
plt.title('Distribution of Document Word Counts', fontdict=dict(size=22))
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.colors as mcolors


In [ ]:
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

fig, axes = plt.subplots(3,2,figsize=(16,14), dpi=160, sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):    
    df_dominant_topic_sub = df_dominant_topic.loc[df_dominant_topic.Dominant_Topic == i, :]
    doc_lens = [len(d) for d in df_dominant_topic_sub.Text]
    ax.hist(doc_lens, bins = 1000, color=cols[i])
    ax.tick_params(axis='y', labelcolor=cols[i], color=cols[i])
    sns.kdeplot(doc_lens, color="black", shade=False, ax=ax.twinx())
    ax.set(xlim=(0, 1000), xlabel='Document Word Count')
    ax.set_ylabel('Number of Documents', color=cols[i])
    ax.set_title('Topic: '+str(i), fontdict=dict(size=16, color=cols[i]))

fig.tight_layout()
fig.subplots_adjust(top=0.90)
plt.xticks(np.linspace(0,1000,9))
fig.suptitle('Distribution of Document Word Counts by Dominant Topic', fontsize=22)
plt.show()

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in merged_content for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(3, 2, figsize=(10,10), sharey=True, dpi= 120)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=1)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()


In [ ]:
# Sentence Coloring of N Sentences
from matplotlib.patches import Rectangle

def sentences_chart(lda_model=lda_model, corpus=corpus, start = 0, end = 13):
    corp = corpus[start:end]
    mycolors = [color for name, color in mcolors.TABLEAU_COLORS.items()]

    fig, axes = plt.subplots(end-start, 1, figsize=(20, (end-start)*0.95), dpi=160)       
    axes[0].axis('off')
    for i, ax in enumerate(axes):
        if i > 0:
            corp_cur = corp[i-1] 
            topic_percs, wordid_topics, wordid_phivalues = lda_model[corp_cur]
            word_dominanttopic = [(lda_model.id2word[wd], topic[0]) for wd, topic in wordid_topics]    
            ax.text(0.01, 0.5, "Doc " + str(i-1) + ": ", verticalalignment='center',
                    fontsize=16, color='black', transform=ax.transAxes, fontweight=700)

            # Draw Rectange
            topic_percs_sorted = sorted(topic_percs, key=lambda x: (x[1]), reverse=True)
            ax.add_patch(Rectangle((0.0, 0.05), 0.99, 0.90, fill=None, alpha=1, 
                                   color=mycolors[topic_percs_sorted[0][0]], linewidth=2))

            word_pos = 0.06
            for j, (word, topics) in enumerate(word_dominanttopic):
                if j < 14:
                    ax.text(word_pos, 0.5, word,
                            horizontalalignment='left',
                            verticalalignment='center',
                            fontsize=16, color=mycolors[topics],
                            transform=ax.transAxes, fontweight=700)
                    word_pos += .009 * len(word)  # to move the word for the next iter
                    ax.axis('off')
            ax.text(word_pos, 0.5, '. . .',
                    horizontalalignment='left',
                    verticalalignment='center',
                    fontsize=16, color='black',
                    transform=ax.transAxes)       

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.suptitle('Sentence Topic Coloring for Documents: ' + str(start) + ' to ' + str(end-2), fontsize=22, y=0.95, fontweight=700)
    plt.tight_layout()
    plt.show()

sentences_chart()  

In [ ]:
# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs, wordid_topics, wordid_phivalues = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=lda_model, corpus=corpus, end=-1)            

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])
dominant_topic_in_each_doc = df.groupby('Dominant_Topic').size()
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in lda_model.show_topics(formatted=False) 
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

from matplotlib.ticker import FuncFormatter

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), dpi=120, sharey=True)

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.5, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax1.set_ylabel('Number of Documents')
ax1.set_ylim(0, 30)


# Topic Distribution by Topic Weights
ax2.bar(x='index', height='count', data=df_topic_weightage_by_doc, width=.5, color='steelblue')
ax2.set_xticks(range(df_topic_weightage_by_doc.index.unique().__len__()))
ax2.xaxis.set_major_formatter(tick_formatter)
ax2.set_title('Number of Documents by Topic Weightage', fontdict=dict(size=10))
ax2.set_ylabel('Number of Documents')
ax1.set_ylim(0, 30)

plt.show()